In [1]:
## ---------------------------------------------------------------------
## set up configs for huggingface hub and OS paths on HPC cluster -- make sure config.ini is correct
## ---------------------------------------------------------------------
import configparser
def auth_token():

    config = configparser.ConfigParser()
    config.read("config.ini")
    return config["hugging_face"]["token"]

def scratch_path():
    config = configparser.ConfigParser()
    config.read("config.ini")
    return "/scratch/" + config["user"]["username"] + "/"

import os
if os.path.isdir(scratch_path()):
    os.environ['TRANSFORMERS_CACHE'] = scratch_path() + '.cache/huggingface'
    os.environ['HF_DATASETS_CACHE'] = scratch_path() + '.cache/huggingface/datasets'
print(os.getenv('TRANSFORMERS_CACHE'))
print(os.getenv('HF_DATASETS_CACHE'))

## ---------------------------------------------------------------------
## Load libraries
## ---------------------------------------------------------------------

import numpy as np
import pandas as pd

import torch
import transformers
from transformers import AutoTokenizer, AutoModel, LlamaForCausalLM, LlamaTokenizer

import torch.nn.functional as F

from entailma import * ## these are where the QA and prompting functions live now
from easyeditor.custom import EditedModel
from easyeditor import LoRAHyperParams

## ---------------------------------------------------------------------
## Ensure GPU is available -- device should == 'cuda'
## ---------------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/scratch/dmpowell/.cache/huggingface
/scratch/dmpowell/.cache/huggingface/datasets


/data/grp_dmpowell/.mamba/envs/EasyEditShared/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/data/grp_dmpowell/.mamba/envs/EasyEditShared/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


device =  cuda


In [2]:
## ---------------------------------------------------------------------
## load llama-2 as a EditedModel class (not pipeline, to integrate better with other scripts/notebooks)
## ---------------------------------------------------------------------

MODEL_NAME = "meta-llama/Llama-2-7b-hf" 

# tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
# model = LlamaForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map = "auto")

hparams = LoRAHyperParams.from_hparams('hparams/LoRA/llama-7b-canonical.yaml')
model = EditedModel(hparams, auth_token())

2024-08-09 17:45:27,824 - easyeditor.editors.editor - INFO - Instantiating model
08/09/2024 17:45:27 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
2024-08-09 17:46:34,483 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
08/09/2024 17:46:34 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...


In [4]:
df = pd.read_csv("data/obqa/test.tsv", sep='\t')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()

df2 = df.copy().tail(10) # smaller df for testing
df2.head(5)

,id,question_stem,choices,complete_question,answer_key
490,9-743,where might a bunny live?,(A) a thicket (B) atop palm trees (C) a sewer ...,where might a bunny live? (A) a thicket (B) at...,A
491,9-645,A shark will be unable to survive on eating al...,(A) it is a predator (B) it is a vegetarian (C...,A shark will be unable to survive on eating al...,A
492,8-250,"A meadow vole just gave birth, and needs to fe...",(A) oil (B) deer (C) bugs (D) recycled plastic...,"A meadow vole just gave birth, and needs to fe...",C
493,283,The Grand Canyon was formed by,(A) a volcano erupting in 1782 (B) a river nam...,The Grand Canyon was formed by (A) a volcano e...,C
494,8-183,"A woman, with a pale complexion, wants to spen...",(A) UV rays are harmful (B) sunlight will be f...,"A woman, with a pale complexion, wants to spen...",A


## ~~answer_questions()~~ mc_choose_answer() function

This function will read a multiple choice question from the dataset and output a single letter response.

In [14]:
df2['model_answer'] = df2.apply(
    lambda row: mc_choose_answer(row['complete_question'], model.model, model.tok),
    axis=1
)


In [15]:
# sum(df2["Answer Key"] == df2["Model Answer"])  

7

This is getting ~58% accuracy. For reference, the original GPT-3 with 32-shot examples got 65.8% ([Brown et al., 2020](https://arxiv.org/abs/2005.14165v4)). So that seems not-too-bad.

## generate_premises() function
~~This function will read the model's statement from the data set and provide two premises that would make the statement true.~~

UPDATE: This seems to work better if we include the original question and answer, which eliminates a point of failure and gives more context for the explanation / premise generation.

UPDATE 2: This is in the `entailma` library in this repo, but I've reproduced it here to make it easier to play around with as you/we tweak prompts.


In [374]:
with open("entailma/prompt3b.txt", 'r') as file:
    premises_prompt = file.read()
    

def generate_premises(question, answer, model, tokenizer):
    
    input_str = f"\n\n{premises_prompt}Question: {question}\nAnswer: {answer}\n"

    pipe = transformers.pipeline(
        "text-generation",
        model = model,
        tokenizer = tokenizer,
        torch_dtype=torch.float16,
        # device_map="cuda",
        device = model.device,
        # use_auth_token = auth_token()
    )

    # sequences = pipe(
    #     input_str,
    #     do_sample=True,
    #     top_p = .95,
    #     # num_beams = 9, # beam search may be better ...
    #     # num_beam_groups = 3,
    #     max_new_tokens = 75,
    #     temperature = 0.7,
    #     num_return_sequences = 5
    # )

    sequences = pipe(
        input_str,
        penalty_alpha=0.9, 
        top_k=5,
        max_new_tokens = 75,
        num_return_sequences = 10
    )
    
    generated_texts = [s['generated_text'] for s in sequences]
    premises = [t[len(input_str):-1] for t in generated_texts]
    premlist = [p.split(".\n")[:2] for p in premises] 

    return premlist if len(premlist) > 1 else premlist[0]


# df2['Generated Premises'] = df2.tail(1).copy().apply(
#     lambda row: generate_premises(row['Complete Question'], row['Answer Key'], model.model, model.tok),
#     axis=1
# )

for row in df2.tail(1).itertuples():
    out = generate_premises(row.complete_question, row.answer_key, model.model, model.tok)
    print(out)

    

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[['Animals use sweat to cool their bodies', 'Sweat is a liquid that evaporates from the skin, which cools the body'], ['Sweat is a liquid that helps animals regulate body temperature', 'Animals use sweat to cool down when they get too hot'], ['Sweat is a liquid produced by the skin that helps regulate body temperature', 'Sweat is produced in response to heat and helps cool the body'], ['Animals use sweat glands to regulate body temperature', 'Sweat contains water and salt, which are lost through evaporation and help cool the body'], ['Animals use sweat to regulate body temperature', 'Sweat is a liquid produced by the skin'], ['Sweat is a liquid produced by the skin', 'Sweat helps animals regulate body temperature'], ['Animals use sweat to regulate their body temperature', 'Sweat is a liquid that is produced by the body to cool it down'], ['Sweat is a liquid that cools the body by evaporating off the skin', 'Sweat is produced in response to heat'], ['Sweat is a liquid produced by the sk

In [103]:
premises_prompt

"Question: Electricity causes less damage to the Earth's atmosphere than (A) Gasoline (B) Potatoes (C) The sun (D) Water\nAnswer: A\nBurning gasoline releases pollutants.\nPollutants damage the Earth's Atmosphere.\n\nQuestion: If a river is rushing southwest on a sunny day, then it is safe to assume that (A) southwest is a good place to be (B) the land gently inclines in that direction (C) the world is mostly land (D) the land is supple\nAnswer: B\nWater flows from a high point to a lower point.\nThe direction of a river is the incline of the land.\n\nQuestion: Using a metal kitchen tool on a cheese can create (A) milk (B) blue cheese (C) melted cheese (D) small pieces\nAnswer: D\nMetal kitchen tools can cut food into small pieces.\nCheese can be easily cut with metal kitchen tools.\n\nQuestion: Poison causes harm to which of the following? (A) a Tree (B) a robot (C) a house (D) a car\nAnswer: A\nPoison causes harm to living things.\nA tree is a living thing.\n\nQuestion: There is most

In [99]:
df2.iloc[5].complete_question

'A person is heating water in order to cook pasta. He spills the pot of water on his leg and finds that the water (A) scalds (B) cools (C) toasts (D) freezes'

In [89]:
def reverse_statement(statement, model, tokenizer):
    
    with open("rephrase-prompt.txt", 'r') as file:
        reverse_prompt = file.read()
    input_str = reverse_prompt + "\n\n[original] " + statement + '\n[reversed]'
    # print(input_str)

    input_len = len(tokenizer(statement)['input_ids'])

    pipe = transformers.pipeline(
        "text-generation",
        model = model,
        tokenizer = tokenizer,
        torch_dtype=torch.float16,
        device = model.device,
    )

    sequences = pipe(
        input_str,
        # do_sample = False,
        penalty_alpha=0.9, 
        top_k=5,
        # num_beams = 5, # beam search may be better ...
        max_new_tokens = input_len + 10
    )
    
    generated_texts = [s['generated_text'] for s in sequences]
    generation = [t[len(input_str):-1] for t in generated_texts]


    return generation


reverse_statement('Soil is composed of mineral particles, organic matter, and microorganisms.', model.model, model.tok)

[' Soil is composed of microorganisms, mineral particles, and organic matter.\n\n[original] A tsunami is ']

In [122]:
def seq_logprob(text, model):
   # here: avged per token
   return(model.completion_logprob(text, text) / len(model.tok(text)['input_ids']))

seq_logprob('Soil is composed of mineral particles, and organic matter', model)

tensor(-3.3723, device='cuda:0')

In [161]:
model.choose(mc_answer_prompt + '\n\nQuestion: A person is heating water in order to cook pasta. He spills the pot of water on his leg and finds that the water (A) scalds (B) cools (C) toasts (D) freezes\nAnswer:', choices = ["A", "B", "C", "D"])

0

In [91]:
# somehow this doesn't work?

model.tok.decode(model.tok('hello world')['input_ids'][1:])

'hello world'

In [281]:

# def premise_logprob():
def logprobs(text, model):
    x = model.logprobs(text)
    tok_idx = x['tokens']['input_ids'].squeeze()
    logprobs = x['logprobs']

    return logprobs[0, :, tok_idx[1:]].squeeze().diag()


def comps_logprobs(text, comp_list, model):
    comp_idx = model.tok(comp_list)
    x = model.logprobs(text)
    tok_idx = x['tokens']['input_ids'].squeeze()
    logprobs = x['logprobs']

    logprobs[0, :, comp_list]



def seq_logprob(text, model, norm = False):
   return logprobs(text, model).sum() if not norm else logprobs(text, model).sum()/(len(model.tok(text)['input_ids']) - 1)


seq_logprob('The sound a dog makes is ', model)

# x['logprobs'][]

# def seq_logprob(text, model):
#    # here: avged per token
#    return(model.completion_logprob(text, text) / len(model.tok(text)['input_ids']))


# Seems like question would be, degree to which is changes odds of original answer vs other answers
# but need this also to work for cases where orig answer was right ?

# def validate_premises():
# def validate_premises(premises, question, answer, model):
   
#    logit_0 = tok_logprobs(mc_answer_prompt +  "\n\nQuestion:" + question+"\nAnswer: " + answer, model)[-1]

#    premise_str = "\n".join(premises)
#    logit_1 = tok_logprobs(mc_answer_prompt + '\n\n' + premise_str + '\nQuestion: ' + question  + 'Answer: ' + answer, model)[-1]

#    # return odds ratio: odd_before (target vs others) / odds_after(target vs others)
#    return(logit_0, logit_1)


# validate_premises(['Some things are red.', 'A fair number of things in the world are red.'], 'What color is the sky? (A) blue (B) red (C) yellow (D) black', 'B', model)
# append the premises ahead of the (raw) question and check change in odds/probability of specified answer

In [414]:
def mc_choose_answer(question, model, tokenizer=None):
    if not tokenizer:
        tokenizer = model.tok
    
    input_str = mc_answer_prompt + f"\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(input_str, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    sequences = model.generate(input_ids = input_ids, max_new_tokens = 1)
    
    return tokenizer.decode(sequences[0])[-1]


def last_token_logprobs(text, last_tokens, model):
    x = model.logprobs(text)
    logprobs = x['logprobs']
    t_idx = [i[-1] for i in model.tok(last_tokens)['input_ids']]

    return(logprobs[0, -1, t_idx])


def mc_answer_logprobs(question, model, answers = ['A','B','C','D']):

    input_str = mc_answer_prompt + f"\n\nQuestion: {question}\nAnswer: "

    return last_token_logprobs(input_str, answers, model)


mc_answer_logprobs('What color is the sky? (A) blue (B) red (C) orange (D) black', model)

tensor([-1.8646, -2.9641, -3.7612, -3.6786], device='cuda:0')

## updates:


- Need a way to score whether the premises are actually any "good" -- i.e. do they lead the model to choose the targeted answer? The code below implements an IKE/ICE-style version of this. It seems to work ok?
- Need to add more examples to the prompt of premises supportin INCORRECT answers, as it struggles with this ATM

In [415]:
def score_premises(premises, question, target_answer, model, answers = ['A','B','C','D']):

   reg_answer_prompt = mc_answer_prompt +  "\n\nQuestion:" + question+"\nAnswer: "
   logprobs0 = last_token_logprobs(reg_answer_prompt, answers, model)
   prob0 = logprobs0[answers.index(target_answer)].exp() / logprobs0.exp().sum()

   premise_str = "\n".join(premises)
   augmented_answer_prompt = mc_answer_prompt + '\n\n' + premise_str + '\nQuestion: ' + question  + 'Answer: '
   logprobs1 = last_token_logprobs(augmented_answer_prompt,  answers, model)
   prob1 = logprobs1[answers.index(target_answer)].exp() / logprobs1.exp().sum()

   return( (prob1/(1-prob1)) / (prob0/(1-prob0)))


In [416]:
# The first are very good premises that reinforce the target answer, the latter are not!

print(score_premises(['The sky is red.', 'At sunset, the sun can be extremely red.'], 'What color is the sky? (A) blue (B) red (C) yellow (D) black', 'B', model))
print(score_premises(['Some things are red.', 'My favorite color is red.'], 'What color is the sky? (A) blue (B) red (C) yellow (D) black', 'B', model))
print(score_premises(['red red red red.', 'red red red red red.'], 'What color is the sky? (A) blue (B) red (C) yellow (D) black', 'B', model))

tensor(2.4303, device='cuda:0')
tensor(1.5171, device='cuda:0')
tensor(1.2276, device='cuda:0')


In [417]:
df2.tail(1).iloc[0].complete_question

'Some animals use a liquid coming from their skin to adjust to (A) cold (B) water (C) heat (D) humidity'

In [430]:
for row in df2.tail(1).itertuples():
    out = generate_premises(row.complete_question, 'C', model.model, model.tok)
    print(out)

for ps in out:
    print(score_premises(ps, df2.tail(1).iloc[0].complete_question, 'C', model))

[['Sweat helps animals regulate their body temperature in hot environments', 'Sweat is a liquid that evaporates and cools the skin'], ['Animals use sweat to cool their bodies', 'Sweat contains water, which evaporates and cools the body'], ['Sweat is a liquid that helps regulate body temperature', 'Sweating helps animals cool down in hot environments'], ['Animals can secrete water from their skin to cool down', 'Water is a liquid'], ['Sweat is a liquid produced by the skin that helps regulate body temperature', 'Sweat helps animals stay cool in hot environments'], ['Animals use sweat to regulate body temperature', 'Sweat is a liquid that evaporates to cool the body'], ['Sweat is a liquid that helps regulate body temperature', 'Sweating helps animals adjust to hot environments'], ['Animals that live in hot and dry environments use sweat to cool their bodies', 'Sweat is a liquid that is secreted from the skin'], ['Sweat is a liquid produced by the skin that helps regulate body temperature

In [440]:
score_premises(['Sweat is a liquid that comes from the skin of some animals.', "Sweat helps animals adjust to hot environments."], df2.tail(1).iloc[0].complete_question, 'C', model)
# out[6]

tensor(1.9020, device='cuda:0')

In [404]:
torch.tensor([-6.3389, -7.1079, -7.1321, -7.1137]).exp() / torch.tensor([-6.3389, -7.1079, -7.1321, -7.1137]).exp().sum(), torch.tensor([-5.1819, -5.4479, -5.6720, -5.6951]).exp() / torch.tensor([-5.1819, -5.4479, -5.6720, -5.6951]).exp().sum()

(tensor([0.4208, 0.1950, 0.1903, 0.1939]),
 tensor([0.3358, 0.2574, 0.2057, 0.2010]))

In [300]:
x = model.logprobs('hello how are you today?')
# tok_idx = x['tokens']['input_ids'].squeeze()

logits = x['logprobs']

logits[0, :, tok_idx[1:]].squeeze().diag()

tensor([-15.9837,  -5.2983,  -9.4110], device='cuda:0')

In [405]:
df2.tail(1).iloc[0].complete_question

'Some animals use a liquid coming from their skin to adjust to (A) cold (B) water (C) heat (D) humidity'

In [196]:
# model.tok.decode(tokens.squeeze()[1:])

# tok_idx[1:]
# logits[0,1:, tok_idx[1:]]
print(tok_idx)
# model.tok.decode(tok_idx[1:])
logits[0][3][tok_idx[3]]

## looks like, 0th place, look at token index for 1st

tensor([    1, 22172,   920,   526,   366,  9826, 29973], device='cuda:0')


tensor(-8.5676, device='cuda:0')

In [351]:
['a','b','c'].index('b')

1